In [ ]:
from glob import glob
from pathlib import Path
from PipeLine import *

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

### This script is used in GNU-Parallel

In [ ]:
from glob import glob
from PipeLine import *

fname = glob("./N*.bug.*")
fname = PipeLine.file_reader(fname) # This is a list with one member

save_to="./"
geom = 'cylindrical'
print(fname)
PipeLine.extract_trj_bug(fname[0], geom, save_to) # A list with one member, the member is a tuple of a trj and data pair.
#PipeLine.bug_trj_rmsd(fname[0], geom, save_to) 

trj_files = glob("./N*all.lammpstrj")
all_tuples = PipeLine.file_reader(trj_files,extensions=['lammpstrj'])
all_trjs = [all_tuple[0] for all_tuple in all_tuples]

data_file = glob("./N*.all.data")
all_data = PipeLine.file_reader(data_file,extensions=['all.data'])
all_data = all_data[0][0]

    
for all_trj in all_trjs:
    print(all_trj)
    PipeLine.extract_trj_all(all_data, all_trj, geom, save_to)

## Standard approach: Running on clusters: extraction from orgaznied *trjs_all* and *trjs_bug* directories

### This are not work properly on Graham cluster but work well on iMacmini

### 1. Extract from an organized *trjs_bug* directory:

In [ ]:
# This script extract different bug's information from pairs (toplogy and trajectory) of bug simulation files in oen or more organized *trjs_bug* directories.
from pathlib import Path
import os
from glob import glob
from PipeLine import *
from dask.distributed import Client
from dask import delayed
from dask import compute

cores = 32
print(f"number of workers set to {cores}; is this the same requested cores on the cluster?")
client = Client(n_workers=cores)
home = str(Path.home())
cwdir = str(Path.cwd())
# information extraction from simulations
geom = 'cylindrical'
fname = glob(home+'/amirhsi_rrg/cylinder_simulations/N*-trjs_bug/N*bug*')
bug_pairs = PipeLine.file_reader(fname) # each bug_pair is a pair of trajectory and topopgy file.
trjs_computed = []
bug_dir = 'extraction_bug/'
for bug_pair in bug_pairs:
    sim_name = bug_pair[0].split("/")[-1].split('bug')[0]
    sim_dir = cwdir+bug_dir+sim_name
    Path(sim_dir).mkdir(parents=True, exist_ok=False)
    sim_save_to = sim_dir+"/"
    trj_delayed = delayed(PipeLine.extract_trj_bug)(bug_pair, geom,sim_save_to)
    trjs_computed.append(trj_delayed)
results = compute(trjs_computed)

### 2. Extract from an organized *trjs_all* directory:

In [ ]:
# This script extract different bug's information from pairs (toplogy and trajectory) of bug simulation files in oen or more organized *trjs_bug* directories.
from pathlib import Path
import os
from glob import glob
from PipeLine import *
from dask.distributed import Client
from dask import delayed
from dask import compute

cores = 32
print(f"number of workers set to {cores}; is this the same requested cores on the cluster?")
client = Client(n_workers=cores)
home = str(Path.home())
cwdir = str(Path.cwd())
sim_all_dirs = glob(home+'/amirhsi_rrg/cylinder_simulations/N*-trjs_all/N*/')
geom = 'cylindrical'

trjs_computed = []
all_extraction_dir = 'extraction_all/'
for sim_all_dir in sim_all_dirs:
    sim_name = sim_all_dir[0].split("/")[-1]
    all_trjs = glob(sim_all_dir+"N*.lammpstrj")
    all_trjs = PipeLine.file_reader(all_trjs,extensions=['lammpstrj'])
    all_trjs = [all_trj[0] for all_trj in all_trjs]

    all_topology = glob(sim_all_dir+"N*.all.data")
    all_topology = PipeLine.file_reader(all_topology,extensions=['all.data'])
    all_topology = all_topology[0][0]
    
    
    sim_extract_dir = cwdir+all_extraction_dir+sim_name
    Path(sim_extract_dir).mkdir(parents=True, exist_ok=False)
    sim_save_to = sim_extract_dir+"/"
    
    for all_trj in all_trjs:
        trj_delayed = delayed(PipeLine.extract_trj_all)(all_topology, all_trj, geom,sim_save_To)
        trjs_computed.append(trj_delayed)

results = compute(trjs_computed)

## Extraction from *extraction_bug* directory after a simulation

In [ ]:
home = str(Path.home())
path=home+'N2000epsilon5.0r10.5lz336sig1.0nc100800dt0.005bdump1000adump5000ens1'
fname = glob(path+"/N*.bug.*")
fname = PipeLine.file_reader(fname) # This is a list with one member
geom = 'cylindrical'
print(fname)
PipeLine.extract_trj_bug(fname[0], geom) # A list with one member, the member is a tuple of a trj and data pair.
PipeLine.rmsd_trj_bug(fname[0], geom)

In [ ]:
# all the segments in one trajectory: M dump files + one data file.
path='/Users/amirhsi_mini/N2000epsilon5.0r10.5lz336sig1.0nc100800dt0.005bdump1000adump5000ens*'
geom = 'cylindrical'
trj_files = glob(path+"/N*.lammpstrj")
all_tuples = PipeLine.file_reader(trj_files,extensions=['lammpstrj',])
all_trjs = [all_tuple[0] for all_tuple in all_tuples]
data_file = glob(path+"/N*.all.data")
all_data = PipeLine.file_reader(data_file,extensions=['all.data'])
all_data = all_data[0][0]
for all_trj in all_trjs:
    PipeLine.extract_trj_all(all_data, all_trj, geom)

## New approach: tested on iMac Pro:

### A single ensemble with one or more segments with one data file

In [ ]:
path='/Users/amirhsi_mini/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens*'
#path='/Users/amirhsi_mini/N1000epsilon5.0r8.0lz308.5sig2.0nc10412dt0.005bdump1000adump5000ens*'
geom = 'cylindrical'
trj_files = glob(path+"/N*all*")
all_pairs = PipeLine.file_reader(trj_files)
trjs_computed = []
for all_pair in all_pairs:
    trj_delayed = delayed(PipeLine.extract_all_trj_polymer_cog_fsd)(all_pair[1], all_pair[0], geom)
    trjs_computed.append(trj_delayed)

In [ ]:
%%time
results = compute(trjs_computed)

### N ensemble with N data file, each ensemble with one or more segments 

In [ ]:
path='/Users/amirhsi_mini/N2000epsilon5.0r15.5lz379.5sig6.0nc1068dt0.005bdump1000adump5000ens*'
#path='/Users/amirhsi_mini/N1000epsilon5.0r8.0lz308.5sig2.0nc10412dt0.005bdump1000adump5000ens*'
geom = 'cylindrical'
trj_pathes = glob(path+"/N*all*")
trjs = PipeLine.file_reader(trj_pathes,extensions=['lammpstrj'])
trjs = [trj[0] for trj in trjs]
topology_pathes = glob(path+"/N*.all.data")
topologies = PipeLine.file_reader(topology_pathes,extensions=['all.data'])
topologies = [topology[0] for topology in topologies]

In [ ]:
def simulation(pair):
    """
    simulation_pair pairs an "all" topology file with all the "all" trjectories of that "all" topology.
    
    Parameters:
    pair (list of tuples): a list in whic each tuple is  pair of topolgy and trajectories of a simulations.
    
    Return:
    a dict of of simulation pairs.
    """
    return {'topology':pair[0], 'trajectories':pair[1]}

In [ ]:
ens_names = [topology.split("/")[-1].split('.all')[0] for topology in topologies]
ens_names = list(dict.fromkeys(ens_names))
trjs_per_ens = []
for ens_name in ens_names:
    ens_trjs = []
    for trj in trjs:
        trj_name = trj.split("/")[-1].split(".all")[0]
        if trj_name == ens_name:
            ens_trjs.append(trj)
            #ensembles[key]['trajectories'] = trj
    trjs_per_ens.append(ens_trjs)
ensembles= dict(zip(ens_names,list(map(simulation,list(zip(topologies,trjs_per_ens))))))

In [ ]:
geom = 'cylindrical'
trjs_computed = []
for ensemble in ensembles.values():
    for trj_segment in ensemble['trajectories']:
        trj_delayed = delayed(PipeLine.extract_trj_all)(ensemble['topology'], trj_segment, geom)
        trjs_computed.append(trj_delayed)

In [ ]:
%%time
results = compute(trjs_computed)